In [ ]:
import pandas as pd
import numpy as np
import re
import time
import schedule
import datetime
import gc

In [ ]:
# neg-1, neu-2, pos-3  
def senti_number(txt):
    if txt == 'Negative':
        txt = re.sub(r'Negative', 1, txt)
    elif txt == 'Neutral':
        txt = re.sub(r'Neutral', 2, txt)
    else:
        txt = re.sub(r'Positive', 3, txt)
    
    return txt

In [ ]:
# UTC -> UNIX
def utc_to_unix(txt):
    utc = datetime.datetime.strptime(txt, '%Y-%m-%dT%H:%M:%S.000Z')
    return utc.timestamp()
    

In [ ]:
# 区間ナンバー割り当て(2021-01-01T00:00を区間0とする)
def section(unix,timespan):
    section = int((unix - 1609426800)/timespan)
    return section

In [ ]:
# pos,neg,newを区間ごとにカウント
def senti_counter(df):    
    return df.value_counts(sort=False).reset_index(name='count')

In [ ]:
# 区間ごとにcomの平均をカウント
def com_counter(df):
    df_groupby = df.groupby("section") 
    return df_groupby.mean()

In [ ]:
# ラベル up, down, stay （始値―終値）
def trend(timespan)

In [ ]:
# main
df = pd.read_csv('tweet-sentiment/2021-01_slist.csv', usecols = ['Compound','created_at'])
#df1 = pd.DataFrame(columns = ['section','neg_count','neu_count','pos_count','com_ave','tweet_count'])
df['created_at'] = df['created_at'].apply(utc_to_unix)
df['created_at'] = df['created_at'].apply(section,args=(24*60*60,))


In [ ]:
df_groupby = df.groupby("created_at") #Name列で集約されたオブジェクトを作成
df_groupby.mean() #Nameのカテゴリー別平均

In [ ]:
tmp_df = df.value_counts(sort=False).reset_index(name='count')

In [ ]:
# メモリ開放
del df,tmp_df
gc.collect()

In [ ]:
print(tmp_df)

In [ ]:
df.head(10)

In [ ]:
print(sorted_df)